<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/google_colab/clip_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install transformers

In [ ]:
from urllib.request import urlopen
from PIL import Image

puppy_path = "https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/chapter09/images/puppy.png"
image = Image.open(urlopen(puppy_path)).convert("RGB")

caption = "a puppy playing in the snow"

In [ ]:
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-base-patch32"

clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)
clip_processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

In [ ]:
inputs = clip_tokenizer(caption, return_tensors="pt")
inputs

In [ ]:
clip_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
# embedding
text_embedding = model.get_text_features(**inputs)
text_embedding.shape

In [ ]:
processed_image = clip_processor(text=None, images=image, return_tensors="pt")[
    "pixel_values"
]
processed_image.shape

In [ ]:
# show image preprocessing
%pip install numpy
%pip install torch
%pip install matplotlib

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

img = processed_image.squeeze(0)
img = img.permute(*torch.arange(img.ndim - 1, -1, -1))
img = np.einsum("ijk->jik", img)
plt.imshow(img)
plt.axis("off")

In [ ]:
# create image embedding
image_embedding = model.get_image_features(processed_image)
image_embedding.shape

In [ ]:
text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

text_embedding = text_embedding.detach().cpu().numpy()
image_embedding = image_embedding.detach().cpu().numpy()

score = np.dot(text_embedding, image_embedding.T)
score